In [1]:
import PhysicalConstants.CODATA2018: c_0
using QuantumCumulants
using CollectiveSpins
using Symbolics
using JLD2
using Unitful

# System

In [2]:
# Nbr of particles
N = 3

# Normalisation parameters
λ = 421e-9
γ = 32.7e6 # In Hz

# Physical constant
ħ = 1. # We can normalize ħ to 1 as no masses are defined in the system

# Physical values
ω0 = (2π*ustrip(c_0)/λ)
ωl = ω0
kl = [ustrip(c_0)/ωl, 0, 0] # Laser along x
Ω_RO = 1e7 # Taken from Barbut arXiv:2412.02541v1

# Position of atoms
Lx, Ly, Lz = [1, 1, 1] * 1e-6
r0 = [[rand(Float64)*Lx, rand(Float64)*Ly, rand(Float64)*Lz] for i in 1:N]

# Normalization
ω0 = ω0 / γ
ωl = ωl / γ
kl = kl * λ
r0 = r0 / λ
Ω_RO = Ω_RO / γ

# Compute the Ω and Γ matrices of the electric dipole-dipole interaction using CollectiveSpins
e = [0, 0, 1.] # Quantization axis along z
system = SpinCollection(r0, e, gammas=1.)
Ω_CS = interaction.OmegaMatrix(system)
Γ_CS = interaction.GammaMatrix(system)

# Integration parameter
tstep = 0.1
T = [0:tstep:10;]; # Normalised time

# Derivation of the symbolic MPC equations

In [3]:
@cnumbers Nsymb ΩROs ω0s

h = NLevelSpace(Symbol(:atom),2)

exp_RO(i) = IndexedVariable(:expRO, i)
conj_exp_RO(i) = IndexedVariable(:conjexpRO, i)
Γ(i,j) = IndexedVariable(:Γ,i,j)
Ω(i,j) = IndexedVariable(:Ω,i,j;identical=false)

i = Index(h,:i,Nsymb,h)
j = Index(h,:j,Nsymb,h)
k = Index(h,:k,Nsymb,h)

σ(x,y,z) = IndexedOperator(Transition(h,:σ,x,y),z)

H0 = ω0s * ∑(-σ(2,2,i), i)
H_RO = ΩROs * ∑(exp_RO(i)*σ(2,1,i) + conj_exp_RO(i)*σ(1,2,i), i)
H_elec = Σ(Ω(i,j)*σ(2,1,i)*σ(1,2,j), j, i)

H = Symbolics.simplify(H0 + H_RO + H_elec)

J = [σ(1,2,i)] # σ-, jump operators for the Lindbladian
rates = [Γ(i,j)]

ops = [σ(2, 2, k), σ(2, 1, k)]; # n_up/σ+

In [4]:
H

(Σ(i=1:Nsymb)-ω0s*(σ22i)+Σ(j=1:Nsymb)(j≠i)Σ(i=1:Nsymb)(i≠j)Ωij*(σ21i*σ12j)+Σ(i=1:Nsymb)expROi*ΩROs*(σ21i)+Σ(i=1:Nsymb)conjexpROi*ΩROs*(σ12i))

In [5]:
eqs = meanfield(ops,H,J;rates=rates,order=2)
complete!(eqs);

### Evaluate equations

In [6]:
eqs_eval = evaluate(eqs; limits=(Nsymb=>N));

### Evaluate parameters

In [7]:
Γij_symb = [Γ(i,j) for i = 1:N for j=1:N]
Ωij_symb = [Ω(i,j) for i = 1:N for j=1:N if i≠j]

Γij_ = [Γ_CS[i, j] for i = 1:N for j=1:N]
Ωij_ = [Ω_CS[i, j] for i = 1:N for j=1:N if i≠j]

exp_RO_symb = [exp_RO(i) for i = 1:N]
conj_exp_RO_symb = [conj_exp_RO(i) for i = 1:N]

exp_RO_ = [exp(1im*r0[i]'kl) for i =1:N]
conj_exp_RO_ = [exp(-1im*r0[i]'kl) for i =1:N]



ps = [Γij_symb; Ωij_symb; exp_RO_symb; conj_exp_RO_; ΩROs; ω0s]
p0 = [Γij_; Ωij_; exp_RO_; conj_exp_RO_; Ω_RO; ω0/2]

p0 = ps .=> p0;

In [11]:
eqs_eval = substitute(eqs_eval, Dict(p0))

∂ₜ(⟨σ221⟩) = (0 - 1im)*((0.04692870622662849 + 0.0im)⟨σ211*σ122⟩ + (-0.049288874239786354 + 0.0im)⟨σ211*σ123⟩) - 0.5((0.14228228979670066 + 0.0im)⟨σ211*σ122⟩ + (0.14228228979670066 + 0.0im)⟨⟨σ211*σ122⟩⟩ + (0.058456253288042666 + 0.0im)⟨σ211*σ123⟩ + (0.058456253288042666 + 0.0im)⟨⟨σ211*σ123⟩⟩) + (7.598331166788746e-15 - 0.3058103975535168im)*⟨σ211⟩ - ⟨σ221⟩ + (0 + 1im)*((0.04692870622662849 + 0.0im)⟨⟨σ211*σ122⟩⟩ + (-0.049288874239786354 + 0.0im)⟨⟨σ211*σ123⟩⟩) + (0.0 + 0.3058103975535168im)*conjexpRO_1*⟨⟨σ211⟩⟩
∂ₜ(⟨σ222⟩) = -0.5((-0.10497216588668129 + 0.0im)⟨⟨σ212*σ123⟩⟩ + (0.14228228979670066 + 0.0im)⟨σ211*σ122⟩ + (-0.10497216588668129 + 0.0im)⟨σ212*σ123⟩ + (0.14228228979670066 + 0.0im)⟨⟨σ211*σ122⟩⟩) + (0 + 1im)*((-0.03352933250992741 + 0.0im)⟨⟨σ212*σ123⟩⟩ + (0.04692870622662849 + 0.0im)⟨σ211*σ122⟩) + (0 - 1im)*((-0.03352933250992741 + 0.0im)⟨σ212*σ123⟩ + (0.04692870622662849 + 0.0im)⟨⟨σ211*σ122⟩⟩) + -⟨σ222⟩ + (1.844827436434393e-14 - 0.3058103975535168im)*⟨σ212⟩ + (0.0 + 0.30581039755

### Compute variables

In [ ]:
# Save the expectation values of the operators which differential equation has to be solved
op_list = []
var_array = []
for i in 1:length(eqs_eval)
    var = eqs_eval[i].lhs
    push!(var_array, var)
    
    v_str = string(var)
    em = eachmatch(r"σ(\d+)", v_str)
    ind = [m.captures[1] for m in em]
    push!(op_list, [parse(Int, i) for i in ind])
end
@save "op_list.jdl2" op_list